In [ ]:
import os
import torch as tc
import numpy as np
from tqdm import tqdm as pgb

from star.star import STAR


from torch.autograd import Variable
from pytorch3d.loss import point_mesh_face_distance
from pytorch3d.structures import Meshes, Pointclouds
from curve_utils import CurveUtils

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

# functions

In [ ]:
def zero_poses(poses):
    poses = poses.clone().detach()
    poses[:,12:17] = 0
    poses[:,54:65] = 0
    poses = Variable(poses, requires_grad=True)
    return poses

def calculate_loss(vertices, faces, scan_meshes, scan_pc):
    star_meshes = Meshes(verts=vertices, faces=faces)
    s2m = point_mesh_face_distance(star_meshes, scan_pc)
    return s2m

# loading bodies

In [ ]:
for gender in ['female', 'male']:

    # loading bodies
    files_path = f"./data/MOVE4D/{gender}/"
    files = os.listdir(files_path)
    files.sort()

    all_vertices = []
    all_faces = []
    for file in pgb(files, desc=f"loading {gender} body"):
        body_vertices = CurveUtils.load_mesh(files_path + file).to(device)
        body_vertices *= 0.001
        body_vertices -= body_vertices.mean(axis=0)
        body_faces, _ = CurveUtils.load_template(files_path + file)
        all_vertices.append(body_vertices)
        all_faces.append(body_faces.to(device))
        
    scan_mesh = Meshes(verts=all_vertices, faces=all_faces)
    scan_pc = Pointclouds(points=all_vertices)

    # loading model
    batch_size = len(all_vertices)
    nbetas=0
    poses = tc.FloatTensor(tc.zeros((batch_size, 72))).to(device)
    poses = Variable(poses, requires_grad=True)
    betas = tc.FloatTensor(tc.zeros((batch_size, nbetas))).to(device)
    betas = Variable(betas, requires_grad=True)
    trans = tc.FloatTensor(tc.zeros((batch_size, 3))).to(device)
    trans = Variable(trans, requires_grad=True)
    star = STAR(gender=gender, num_betas=nbetas)
    star_faces = np.repeat(star.faces[None,...],batch_size,axis=0).to(device)

    # training model
    learning_rate = 0.02
    epochs = 10000
    loop = pgb(range(epochs), desc=f"optimizing {gender} - loss: inf")
    for it in loop:
        
        if it == 20:
            poses = zero_poses(poses)

        if it%300 == 0 and it < 4800:
            nbetas += 20
            new_betas = tc.FloatTensor(np.zeros((batch_size, nbetas))).to(device)
            new_betas[:,:nbetas-20] = betas
            betas = Variable(new_betas, requires_grad=True)
            star = STAR(gender=gender, num_betas=nbetas)
            optimizer = tc.optim.Adam([trans, betas, poses], lr=learning_rate)
        
        d = star(betas=betas, pose=poses, trans=trans)
        optimizer.zero_grad()
        loss = calculate_loss(d, star_faces, scan_mesh,scan_pc)
        loss.backward(retain_graph=True)
        optimizer.step()
        loop.set_description(f"optimizing {gender} - loss: {loss}")
        if loss < 2e-5:
            break

    np.save(f"{gender}_betas.npy", betas.cpu().detach().numpy())
    np.save(f"{gender}_poses.npy", poses.cpu().detach().numpy())
    np.save(f"{gender}_trans.npy", trans.cpu().detach().numpy())